# 第5章 Python中常见的数据结构 

## 5.1 字典、映射和散列表 

在 Python 中，字典是核心数据结构。字典可以存储任意数量的对象，每个对象都由唯一的 字典键标识。 

字典通常也被称为映射、散列表、查找表或关联数组。字典能够高效查找、插入和删除任何 与给定键关联的对象。 

这在现实中意味着什么呢？字典对象相当于现实世界中的电话簿。 
电话簿有助于快速检索与给定键（人名）相关联的信息（电话号码）。因此不必 为了查找某人的号码而浏览整本电话簿，根据人名基本上就能直接跳到需要查找的相关 信息。 

若想研究以何种方式组织信息才有利于快速检索，上述类比就不那么贴切了。但基本性能特 征相同，即字典能够用来快速查找与给定键相关的信息。 

总之，字典是计算机科学中常用且重要的数据结构之一。 
那么 Python如何处理字典呢？ 
我们来看看 Python及其标准库中可用的字典实现。 

5.1.1 dict——首选字典实现 

由于字典非常重要，因此Python直接在语言核心中实现了一个稳健的字典①：dict 数据类型②。

Python还提供了一些有用的“语法糖”来处理程序中的字典。例如，用花括号字典表达式语 法和字典解析式能够方便地创建新的字典对象： 

In [3]:
phonebook = {
    'bob':7387,
    'alice':3719,
    'jack':7052,
}

In [4]:
squares = {x: x*x for x in range(6)}

In [5]:
phonebook['alice']

3719

In [7]:
squares

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16, 5: 25}

关于哪些对象可以作为字典键，有一些限制。 

Python 的字典由可散列类型①的键来索引。可散列对象具有在其生命周期中永远不会改变的 散列值（参见__hash__），并且可以与其他对象进行比较（参见__eq__）。另外，相等的可散列 对象，其散列值必然相同。 

像字符串和数这样的不可变类型是可散列的，它们可以很好地用作字典键。元组对象也可以 用作字典键，但这些元组本身必须只包含可散列类型。 

Python 的内置字典实现可以应对大多数情况。字典是高度优化的，并且是 Python 语言的基 石，例如栈帧中的类属性和变量都存储在字典中

Python字典基于经过充分测试和精心调整过的散列表实现，提供了符合期望的性能特征。一 般情况下，用于查找、插入、更新和删除操作的时间复杂度都为 O(1)。 

大部分情况下，应该使用 Python自带的标准字典实现。但是也存在专门的第三方字典实现， 例如跳跃表②或基于 B树的字典。 

除了通用的 dict 对象外，Python的标准库还包含许多特殊的字典实现。它们都基于内置的 字典类，基本性能特征相同，但添加了其他一些便利特性。 
下面来逐个了解一下。 

### 5.1.2 collections.OrderedDict——能记住键的插入顺序 

collections.OrderedDict③是特殊的 dict 子类，该类型会记录添加到其中的键的插入 顺序。 

尽管在 CPython 3.6 及更高版本中，标准的字典实现也能保留键的插入顺序，但这只是 CPython实现的一个副作用，直到 Python 3.7才将这种特性固定下来了。①因此，如果在自己的工 作中很需要用到键顺序，好明确使用 OrderedDict 类。 

顺便说一句，OrderedDict 不是内置的核心语言部分，因此必须从标准库中的 collections 模块导入。 

In [8]:
import collections

In [10]:
d = collections.OrderedDict(one=1,two=2,three=3)

In [11]:
d

OrderedDict([('one', 1), ('two', 2), ('three', 3)])

In [13]:
d['four'] = 4

In [14]:
d

OrderedDict([('one', 1), ('two', 2), ('three', 3), ('four', 4)])

In [15]:
d.keys()

odict_keys(['one', 'two', 'three', 'four'])

### 5.1.3 collections.defaultdict——为缺失的键返回默认值 

defaultdict 是另一个 dict 子类，其构造函数接受一个可调用对象，查找时如果找不到 给定的键，就返回这个可调用对象。② 

与使用 get()方法或在普通字典中捕获 KeyError 异常相比，这种方式的代码较少，并能 清晰地表达出程序员的意图。 

In [16]:
from collections import defaultdict

In [18]:
dd = defaultdict(list)
#访问缺失的键就会用默认工厂方法创建它并将其初始化
#在本例中工厂方法为list():
dd['dogs'].append('Rufus')
dd['dogs'].append('Kathrin')
dd['dogs'].append('Mr Sniffles')
dd

defaultdict(list, {'dogs': ['Rufus', 'Kathrin', 'Mr Sniffles']})

In [20]:
dd['cat']#利用工厂函数list对其进行初始化

[]

### 5.1.4 collections.ChainMap——搜索多个字典 

collections.ChainMap 数据结构将多个字典分组到一个映射中③，在查找时逐个搜索底层映射，直到找到一个符合条件的键。对 ChainMap 进行插入、更新和删除操作，只会作用于其中 的第一个字典。  

In [21]:
from collections import ChainMap
dict1 = {'one':1,'two':2}
dict2 = {'three':3,'four':4}
chain = ChainMap(dict1,dict2)

In [22]:
chain

ChainMap({'one': 1, 'two': 2}, {'three': 3, 'four': 4})

In [23]:
#ChainMap在内部从左到右逐个搜索
#直到找到对应的键或全部搜索完毕

In [24]:
chain['three']

3

In [25]:
chain['missing']

KeyError: 'missing'

### 5.1.5 types.MappingProxyType——用于创建只读字典 

MappingProxyType 封装了标准的字典，为封装的字典数据提供只读视图。①该类添加自 Python 3.3，用来创建字典不可变的代理版本。 

举例来说，如果希望返回一个字典来表示类或模块的内部状态，同时禁止向该对象写入内容， 此时 MappingProxyType 就能派上用场。使用 MappingProxyType 无须创建完整的字典副本。
 

In [26]:
from types import MappingProxyType
writable = {'one':1,'two':2}
read_only = MappingProxyType(writable)

In [27]:
#代理是只读的
read_only['one']

1

In [28]:
read_only['one'] = 23

TypeError: 'mappingproxy' object does not support item assignment

In [29]:
#更新原字典也会受到影响
writable['one'] = 42

In [30]:
read_only

mappingproxy({'one': 42, 'two': 2})

### 5.1.6 Python中的字典：总结 

本节列出的所有 Python字典实现都是内置于 Python标准库中的有效实现。 

一般情况下，建议在自己的程序中使用内置的 dict 数据类型。这是优化过的散列表实现， 功能多且已被直接内置到了核心语言中。 

如果你有内置 dict 无法满足的特殊需求，那么建议使用本节列出的其他数据类型。 

虽然前面列出的其他字典实现均可用，但大多数情况下都应该使用Python内置的标准dict， 这样其他开发者在维护你的代码时就会轻松一点。 

### 5.1.7 关键要点 

 字典是 Python中的核心数据结构。 

 大部分情况下，内置的 dict 类型就足够了。 

 Python标准库提供了用于满足特殊需求的实现，比如只读字典或有序字典。 

## 5.2 数组数据结构 

大多数编程语言中都有数组这种基本数据结构，它在许多算法中都有广泛的运用。 
本节将介绍 Python中的一些数组实现，这些数组只用到了语言的核心特性或 Python标准库 包含的功能。 

本章还会介绍每种实现的优缺点，这样就能根据实际情况选择合适的实现。不过在介绍之前， 先来了解一些基础知识。 

首先要知道数组的原理及用途。 

数组由大小固定的数据记录组成，根据索引能快速找到其中的每个元素。 

因为数组将信息存储在依次连接的内存块中，所以它是连续的数据结构（与链式列表等链式 数据结构不同）。

现实世界中能用来类比数组数据结构的是停车场。 

停车场可被视为一个整体，即单个对象，但停车场内的每个停车位都有唯一的编号 索引。停车位是车辆的容器，每个停车位既可以为空，也可以停有汽车、摩托车或其他 车辆。 

各个停车场之间也会有区别。 

有些停车场可能只能停一种类型的车辆。例如，汽车停车场不允许停放自行车。 这种“有限制”的停车场相当于“类型数组”数据结构，只允许存储相同数据类型的 元素。 

在性能方面，根据元素的索引能快速查找数组中对应的元素。合理的数组实现能够确保索引 访问的耗时为常量时间 O(1)。 
Python标准库包含几个与数组相似的数据结构，每个数据结构的特征略有不同。下面来逐一 介绍。 

### 5.2.1 列表——可变动态数组 

列表是 Python语言核心的一部分。①虽然名字叫列表，但它实际上是以动态数组实现的。这 意味着列表能够添加或删除元素，还能分配或释放内存来自动调整存储空间。 

Python 列表可以包含任意元素，因为 Python 中一切皆为对象，连函数也是对象。因此，不 同的数据类型可以混合存储在一个列表中。 

这个功能很强大，但缺点是同时支持多种数据类型会导致数据存储得不是很紧凑。因此整个 结构占据了更多的空间。② 

In [39]:
arr = ['one','two','three']
arr[0]

'one'

In [40]:
#列表拥有不错的__repr__方法
arr

['one', 'two', 'three']

In [41]:
#列表是可变的
arr[1] = 'hello'
arr

['one', 'hello', 'three']

In [42]:
del arr[1]
arr

['one', 'three']

In [43]:
#列表可以含有任意类型的数据
arr.append(23)
arr

['one', 'three', 23]

### 5.2.2 元组——不可变容器 

与列表一样，元组也是 Python语言核心的一部分。③与列表不同的是，Python的元组对象是不可变的。这意味着不能动态添加或删除元素，元组中的所有元素都必须在创建时定义。  

就像列表一样，元组可以包含任意数据类型的元素。这具有很强的灵活性，但也意味着数据 的打包密度要比固定类型的数组小。 

In [44]:
arr = 'one','two','three'
arr[0]

'one'

In [45]:
#元组拥有不错的__repr__方法
arr

('one', 'two', 'three')

In [46]:
#元组是不可变的
arr[1] = 'hello'

TypeError: 'tuple' object does not support item assignment

In [47]:
del arr[1]

TypeError: 'tuple' object doesn't support item deletion

In [53]:
#元组可以持有任意类型的数据
#(添加元素会创建新元组)
b = arr+(23,)
b

('one', 'two', 'three', 23)

In [54]:
b is arr

False

In [55]:
arr is arr

True

### 5.2.3 array.array——基本类型数组 

Python的 array 模块占用的空间较少，用于存储 C语言风格的基本数据类型（如字节、32 位整数，以及浮点数等）。

使用 array.array 类创建的数组是可变的，行为与列表类似。但有一个重要的区别：这种 数组是单一数据类型的“类型数组”。

由于这个限制，含有多个元素的 array.array 对象比列表和元组节省空间。存储在其中的 元素紧密排列，因此适合存储许多相同类型的元素。 

此外，数组中有许多普通列表中也含有的方法，使用方式也相同，无须对应用程序代码进行 其他更改。 

In [58]:
import array
arr = array.array('f',(1.0,1.5,2.0,2.5))
arr[1]

1.5

In [59]:
#数组拥有不错的__repr__方法
arr

array('f', [1.0, 1.5, 2.0, 2.5])

In [60]:
#数组是可变的
arr[1] = 23.0
arr

array('f', [1.0, 23.0, 2.0, 2.5])

In [61]:
del arr[1]
arr

array('f', [1.0, 2.0, 2.5])

In [63]:
arr.append(42.0)
arr

array('f', [1.0, 2.0, 2.5, 42.0, 42.0])

In [64]:
# 数组中元素类型是固定的： 
arr[1] = 'hello'

TypeError: must be real number, not str

### 5.2.4 str——含有 Unicode字符的不可变数组 

Python 3.x使用 str 对象将文本数据存储为不可变的 Unicode字符序列。①实际上，这意味着 str 是不可变的字符数组。说来也怪，str 也是一种递归的数据结构，字符串中的每个字符都是 长度为 1的 str 对象

由于字符串对象专注于单一数据类型，元组排列紧密，因此很节省空间，适合用来存储 Unicode文本。因为字符串在 Python中是不可变的，所以修改字符串需要创建一个改动副本。 接近“可变字符串”概念的是存储单个字符的列表。 

In [65]:
arr  = 'abcd'
arr[1]

'b'

In [66]:
arr[1]

'b'

In [67]:
#字符串是不可变的
arr[1] = 'e'

TypeError: 'str' object does not support item assignment

In [68]:
del arr[1]

TypeError: 'str' object doesn't support item deletion

In [69]:
#字符串可以解包到列表中，从而得到可变版本：
list('abcd')

['a', 'b', 'c', 'd']

In [70]:
''.join(list('abcd'))

'abcd'

In [75]:
#字符串是递归型数据类型
print(type('abc'))

<class 'str'>


In [76]:
print(type('abc'[0]))

<class 'str'>


### 5.2.5 bytes——含有单字节的不可变数组 

bytearray 类型是可变整数序列②，包含的整数范围在 0～255（含）。bytearray 与 bytes 对象关系密切，主要区别在于 bytearray 可以自由修改，如覆盖、删除现有元素和添加新元素， 此时 bytearray 对象将相应地增长和缩小。 

bytearray 数可以转换回不可变的 bytes 对象，但是这需要复制所存储的数据，是耗时为 O(n)的慢操作

In [77]:
arr = bytearray((0,1,2,3))
arr[1]

1

In [78]:
#bytearray的repr:
arr

bytearray(b'\x00\x01\x02\x03')

In [80]:
#bytearray是可变的
arr[1] = 23
arr

bytearray(b'\x00\x17\x02\x03')

In [81]:
arr[1]

23

In [82]:
# bytearray 可以增长或缩小： 
del arr[1]
arr

bytearray(b'\x00\x02\x03')

In [83]:
arr.append(42)
arr

bytearray(b'\x00\x02\x03*')

In [84]:
# bytearray 只能持有 byte，即位于 0～255 范围内的整数 
arr[1] = 'hello'

TypeError: an integer is required

In [85]:
arr[1] = '300'

TypeError: an integer is required

In [87]:
# bytearray 可以转换回 byte 对象，此过程会复制数据： 
bytes(arr)

b'\x00\x02\x03*'

### 5.2.7 关键要点 

Python中有多种内置数据结构可用来实现数组，本节只专注位于标准库中和核心语言特性中 的数据结构。 

对于 Python中包含的数组数据结构，选择顺序可归结如下。 

__如果需要存储任意对象，且其中可能含有混合数据类型__，那么可以选择使用列表或元组，前 者可变后者不可变。 

如果存储数值（整数或浮点数）数据并要求排列紧密且注重性能，那么先尝试 array.array， 看能否满足要求。另外可尝试准库之外的软件包，如 NumPy或 Pandas

如果有需要用 Unicode 字符表示的文本数据，那么可以使用 Python内置的 str。如果需要 用到“可变字符串”，则请使用字符列表。 

如果想存储一个连续的字节块，不可变的请使用 bytes，可变的请使用 bytearray。 

总之，在大多数情况下首先应尝试列表。如果在性能或存储空间上有问题，再选择其他专门 的数据类型。一般像列表这样通用的数组型数据结构已经能同时兼顾开发速度和编程便利性的 要求了。 
强烈建议在初期使用通用数据格式，不要试图在一开始就榨干所有性能。 

## 5.3 记录、结构体和纯数据对象 

与数组相比，记录数据结构中的字段数目固定，每个都有一个名称，类型也可以不同。 

本节将介绍 Python中的记录、结构体，以及“纯数据对象”①，但只介绍标准库中含有的内 置数据类型和类。 

顺便说一句，这里的“记录”定义很宽泛。例如，这里也会介绍像 Python 的内置元组这样 的类型。由于元组中的字段没有名称，因此一般不认为它是严格意义上的记录。 

Python提供了几种可用于实现记录、结构体和数据传输对象的数据类型。本节将快速介绍每 个实现及各自特性，后进行总结并给出一个决策指南，用来帮你做出自己的选择。 
好吧，让我们开始吧！ 

### 5.3.1 字典——简单数据对象 

Python 字典能存储任意数量的对象，每个对象都由唯一的键来标识。②字典也常常称为映射 或关联数组，能高效地根据给定的键查找、插入和删除所关联的对象。 

Python的字典还可以作为记录数据类型（record data type）或数据对象来使用。在 Python中 创建字典很容易，因为语言内置了创建字典的语法糖，简洁又方便

字典创建的数据对象是可变的，同时由于可以随意添加和删除字段，因此对字段名称几乎没 有保护措施。这些特性综合起来可能会引入令人惊讶的 bug，毕竟要在便利性和避免错误之间做 出取舍

In [91]:
car1 = {
    'color':'red',
    'mileage':3812.4,
    'automatic':True,
}

car2 = {
    'color':'bule',
    'mileage':40231,
    'automatic':False,
}

In [92]:
#字典有不错的__repr __方法
car2

{'color': 'bule', 'mileage': 40231, 'automatic': False}

In [93]:
#获取mileage
car2['mileage']

40231

In [95]:
#字典是可变的
car2['mileage'] = 12
car2['windshield'] = 'broken'
car2

{'color': 'bule', 'mileage': 12, 'automatic': False, 'windshield': 'broken'}

In [96]:
# 对于提供错误、缺失和额外的字段名称并没有保护措施： 
car3 = {
    'color':'green',
    'automatic':False,
    'windshield':'broken'
}

### 5.3.2 元组——不可变对象集合 

Python 元组是简单的数据结构，用于对任意对象进行分组。①元组是不可变的，创建后无法 修改。 

在性能方面，元组占用的内存略少于 CPython中的列表②，构建速度也更快。 

从如下反汇编的字节码中可以看到，构造元组常量只需要一个 LOAD_CONST 操作码，而构
造具有相同内容的列表对象则需要多个操作： 

In [97]:
import dis
dis.dis(compile("(23,'a','b','c')",'','eval'))

  1           0 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              2 RETURN_VALUE


In [98]:
dis.dis(compile("[23, 'a', 'b', 'c']", '', 'eval')) 

  1           0 LOAD_CONST               0 (23)
              2 LOAD_CONST               1 ('a')
              4 LOAD_CONST               2 ('b')
              6 LOAD_CONST               3 ('c')
              8 BUILD_LIST               4
             10 RETURN_VALUE


不过你无须过分关注这些差异。在实践中这些性能差异通常可以忽略不计，试图通过用元组 替换列表来获得额外的性能提升一般都是入了歧途。 

单纯的元组有一个潜在缺点，即存储在其中的数据只能通过整数索引来访问，无法为元组中 存储的单个属性制定一个名称，从而影响了代码的可读性。

这样很容易因疏忽而犯错，比如弄错字段顺序。因此，建议尽可能减少元组中存储的字段 数量。 

In [99]:
# 字段：color、mileage、automatic 
car1 = ('red',3812.4,True)
car2 = ('bule',40231.0,False)

In [100]:
# 元组的实例有不错的__repr__方法： 
car1

('red', 3812.4, True)

In [101]:
car2

('bule', 40231.0, False)

In [102]:
# 获取 mileage：
car2[1]

40231.0

In [103]:
# 元组是不可变的
car2[1] = 12

TypeError: 'tuple' object does not support item assignment

In [105]:
# 对于错误或额外的字段，以及提供错误的字段顺序，并没有报错措施：
car3 = (3431.5, 'green', True, 'silver') 

### 5.3.3 编写自定义类——手动精细控制 

类可用来为数据对象定义可重用的“蓝图”（blueprint），以确保每个对象都提供相同的字段。
 

普通的 Python 类可作为记录数据类型，但需要手动完成一些其他实现中已有的便利功能。 例如，向__init__构造函数添加新字段就很烦琐且耗时。 

此外，对于从自定义类实例化得到的对象，其默认的字符串表示形式没什么用。解决这个问 题需要添加自己的__repr__方法。①这个方法通常很冗长，每次添加新字段时都必须更新。 

存储在类上的字段是可变的，并且可以随意添加新字段。使用@property 装饰器②能创建只 读字段，并获得更多的访问控制，但是这又需要编写更多的胶水代码。 

编写自定义类适合将业务逻辑和行为添加到记录对象中，但这意味着这些对象在技术上不再 是普通的纯数据对象。 

In [106]:
class Car:
    def __init__(self,color,mileage,automatic):
        self.color = color
        self.mileage = mileage
        self.automatic = automatic

In [107]:
car1 = Car('red',3812.4,True)
car2 = Car('bule',40231.0,False)

In [108]:
#获取mileage
car2.mileage

40231.0

In [110]:
#类是可变的
car2.mileage = 12
car2.windsheild = 'broken'

In [113]:
car2.mileage

12

In [114]:
car2.windsheild

'broken'

In [115]:
# 类的默认字符串形式没多大用处，必须手动编写一个__repr__方法： 
car1

In [118]:
class Car:
    def __init__(self,color,mileage,automatic):
        self.color = color
        self.mileage = mileage
        self.automatic = automatic
    def __repr__(self):
        return (f'{self.__class__.__name__}('
    f'{self.color!r},{self.mileage!r})')

In [119]:
car2 = Car('bule',40231.0,False)

In [120]:
car2

Car('bule',40231.0)

### 5.3.4 collections.namedtuple——方便的数据对象 

自 Python 2.6 以来添加的 namedtuple 类扩展了内置元组数据类型。③与自定义类相似， namedtuple 可以为记录定义可重用的“蓝图”，以确保每次都使用正确的字段名称。 

与普通的元组一样，namedtuple是不可变的。这意味着在创建 namedtuple实例之后就不能再 添加新字段或修改现有字段。 

除此之外，namedtuple就相当于具有名称的元组。存储在其中的每个对象都可以通过唯一标 识符访问。因此无须整数索引，也无须使用变通方法，比如将整数常量定义为索引的助记符。 

namedtuple对象在内部是作为普通的 Python类实现的，其内存占用优于普通的类，和普通元 组一样高效： 

In [122]:
from collections import namedtuple
from sys import getsizeof

p1 = namedtuple('Point','x y z')(1,2,3)
p2 = (1,2,3)

In [123]:
getsizeof(p1)

72

In [124]:
getsizeof(p2)

72

由于使用 namedtuple就必须更好地组织数据，因此无意中清理了代码并让其更加易读。 

我发现从专用的数据类型（例如固定格式的字典）切换到 namedtuple有助于更清楚地表达代 码的意图。通常，每当我在用 namedtuple重构应用时，都神奇地为代码中的问题想出了更好的解 决办法。 

用 namedtuple替换普通（非结构化的）元组和字典还可以减轻同事的负担，因为用namedtuple 传递的数据在某种程度上能做到“自说明”。 

In [125]:
from collections import namedtuple
Car = namedtuple('Car','color mileage automatic')

In [126]:
car1 = Car('red',3812.4,True)

In [127]:
# 实例有不错的__repr__方法： 
car1

Car(color='red', mileage=3812.4, automatic=True)

In [128]:
# 访问字段：
car1.mileage

3812.4

In [129]:
#字段是不可变的
car1.mileage = 12

AttributeError: can't set attribute

In [130]:
car1.windshield = 'broken' 

AttributeError: 'Car' object has no attribute 'windshield'

### 5.3.5 typing.NamedTuple——改进版 namedtuple 

这个类添加自Python 3.6，是collections 模块中 namedtuple 类的姊妹。 ①它与 namedtuple 非常相似，主要区别在于用新语法来定义记录类型并支持类型注解（type hint）。 

注意，只有像mypy这样独立的类型检查工具才会在意类型注解。不过即使没有工具支持，类 型注解也可帮助其他程序员更好地理解代码（如果类型注解没有随代码及时更新则会带来混乱）。 

In [132]:
from typing import NamedTuple
class Car(NamedTuple):
    color:str
    mileage:float
    automatic:bool

In [133]:
car1 = Car('red',3812.4,True)

In [134]:
#实例有不错的__repr__方法:
car1

Car(color='red', mileage=3812.4, automatic=True)

In [135]:
# 访问字段：
car1.mileage

3812.4

In [136]:
# 字段是不可变的： 
car1.mileage = 12

AttributeError: can't set attribute

In [137]:
# 只有像 mypy 这样的类型检查工具才会落实类型注解：
Car('red','Not_A_Float',99)

Car(color='red', mileage='Not_A_Float', automatic=99)

### 5.3.6 struct.Struct——序列化 C结构体 

struct.Struct 类①用于在 Python值和 C结构体之间转换，并将其序列化为 Python字节对 象。例如可以用来处理存储在文件中或来自网络连接的二进制数据。 

结构体使用与格式化字符串类似的语法来定义，能够定义并组织各种 C数据类型（如 char、 int、long，以及对应的无符号的变体）。 

序列化结构体一般不用来表示只在 Python 代码中处理的数据对象，而是主要用作数据交换 格式。 

在某些情况下，与其他数据类型相比，将原始数据类型打包到结构体中占用的内存较少。但 大多数情况下这都属于高级（且可能不必要的）优化

In [138]:
from struct import Struct
MyStruct = Struct('i?f')
data = MyStruct.pack(23,False,42.0)

In [139]:
# 得到的是一团内存中的数据：
data

b'\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00(B'

In [141]:
#数据可以再次解包：
MyStruct.unpack(data)

(23, False, 42.0)

### 5.3.7 types.SimpleNamespace——花哨的属性访问 

这里再介绍一种高深的方法来在 Python中创建数据对象：types.SimpleNamespace①。该 类添加自 Python 3.3，可以用属性访问的方式访问其名称空间。 

也就是说，SimpleNamespace 实例将其中的所有键都公开为类属性。因此访问属性时可以 使用 obj.key 这样的点式语法，不需要用普通字典的 obj['key']方括号索引语法。所有实例 默认都包含一个不错的__repr__。 

正如其名，SimpleNamespace 很简单，基本上就是扩展版的字典，能够很好地访问属性并 以字符串打印出来，还能自由地添加、修改和删除属性。 

In [142]:
from types import SimpleNamespace
car1 = SimpleNamespace(color = 'red',
                      mileage = 3812.4,
                      automatic = True)

In [143]:
# 默认的__repr__效果
car1

namespace(automatic=True, color='red', mileage=3812.4)

In [144]:
# 实例支持属性访问并且是可变的：
car1.mileage = 12
car1.windshield = 'broken'
del car1.automatic
car1

namespace(color='red', mileage=12, windshield='broken')

### 5.3.8 关键要点 

如果只有两三个字段，字段顺序易于记忆或无须使用字段名称，则使用简单元组对象。例三维空间中的(x, y, z)点。 

如果需要实现含有不可变字段的数据对象，则使用 collections.namedtuple 或 typing.NamedTuple 这样的简单元组。 

如果想锁定字段名称来避免输入错误，同样建议使用 collections.namedtuple 和typing. NamedTuple。 

如果希望保持简单，建议使用简单的字典对象，其语法方便，和 JSON也类似。 

如果需要对数据结构完全掌控，可以用@property 加上设置方法和获取方法来编写自定义 的类。

如果需要向对象添加行为（方法），则应该从头开始编写自定义类，或者通过扩展 collections. namedtuple 或 typing.NamedTuple 来编写自定义类。 

如果想严格打包数据以将其序列化到磁盘上或通过网络发送，建议使用 struct.Struct。 
一般情况下，如果想在 Python 中实现一个普通的记录、结构体或数据对象，我的建议是在 Python 2.x中使用collections.namedtuple，在Python 3中使用其姊妹typing.NamedTuple。
 

## 5.4 集合和多重集合 

本节将用标准库中的内置数据类型和类在 Python 中实现可变集合、不可变集合和多重集合 （背包）数据结构。首先来快速回顾一下集合数据结构。 

集合含有一组不含重复元素的无序对象。集合可用来快速检查元素的包含性，插入或删除值， 计算两个集合的并集或交集。 

在“合理”的集合实现中，成员检查预计耗时为 O(1)。并集、交集、差集和子集操作应平均 耗时为 O(n)。Python标准库中的集合实现都具有这些性能指标。①

与字典一样，集合在 Python 中也得到了特殊对待，有语法糖能够方便地创建集合。例如， 花括号集合表达式语法和集合解析式能够方便地定义新的集合实例： 

In [145]:
vowels = {'a','e','i','o','u'}
squares = {x*x for x in range(10)}

但要小心，创建空集时需要调用 set()构造函数。空花括号{}有歧义，会创建一个空字典。
 
Python及其标准库提供了几个集合实现，让我们看看。 

### 5.4.1 set——首选集合实现 

set 是 Python中的内置集合实现。①set 类型是可变的，能够动态插入和删除元素

Python 的集合由 dict 数据类型支持，具有相同的性能特征。所有可散列②的对象都可以存 储在集合中。 

In [147]:
vowels = {'a','e','i','o','u'}

In [150]:
'e' in vowels

True

In [151]:
letters = set('alice')

In [152]:
letters.intersection(vowels)

{'a', 'e', 'i'}

In [153]:
vowels.add('x')

In [154]:
vowels

{'a', 'e', 'i', 'o', 'u', 'x'}

In [155]:
len(vowels)

6

### 5.4.2 frozenset——不可变集合 

frozenset 类实现了不可变版的集合，即在构造后无法更改。③不可变集合是静态的，只能 查询其中的元素（无法插入或删除）。因为不可变集合是静态的且可散列的，所以可以用作字典 的键，也可以放置在另一个集合中，普通可变的 set 对象做不到这一点。 

In [156]:
vowels = frozenset({'a','e','i','o','u'})
vowels.add('p')

AttributeError: 'frozenset' object has no attribute 'add'

In [157]:
# 不可变集合是可散列的，可用作字典的键 
d = {frozenset({1,2,3}):'hello'}

In [158]:
d[frozenset({1,2,3})]

'hello'

### 5.4.3 collections.Counter——多重集合 

Python标准库中的 collections.Counter 类实现了多重集合（也称背包，bag）类型，该 类型允许在集合中多次出现同一个元素。④ 

如果既要检查元素是否为集合的一部分，又要记录元素在集合中出现的次数，那么就需要用 到这个类型

In [165]:
from collections import Counter
inventory = Counter()

loot = {'sword':1,'bread':3}
inventory.update(loot)
inventory

Counter({'sword': 1, 'bread': 3})

In [166]:
more_loot = {'sword':1,'apple':1}
inventory.update(more_loot)
inventory

Counter({'sword': 2, 'bread': 3, 'apple': 1})

Counter 类有一点要注意，在计算 Counter 对象中元素的数量时需要小心。调用 len()返 回的是多重集合中唯一元素的数量，而想获取元素的总数需要使用 sum 函数

In [167]:
len(inventory)

3

In [168]:
sum(inventory.values())

6

### 5.4.4 关键要点 

 集合是 Python及其标准库中含有的另一种有用且常用的数据结构。 

 查找可变集合时可使用内置的 set 类型。 

 frozenset 对象可散列且可用作字典和集合的键。 

 collections.Counter 实现了多重集合或“背包”类型的数据。 

## 5.5 栈（后进先出） 

栈是含有一组对象的容器，支持快速后进先出（LIFO）的插入和删除操作。与列表或数组不 同，栈通常不允许随机访问所包含的对象。插入和删除操作通常称为入栈（push）和出栈（pop）。
 

现实世界中与栈数据结构相似的是一叠盘子。 

新盘子会添加到栈的顶部。由于这些盘子非常宝贵且很重，所以只能移动最上面的 盘子（后进先出）。要到达栈中位置较低的盘子，必须逐一移除最顶端的盘子。 

栈和队列相似，都是线性的元素集合，但元素的访问顺序不同。 


从队列删除元素时，移除的是先添加的项（先进先出，FIFO）；而栈是移除近添加的项 （后进先出，LIFO）。 

在性能方面，合理的栈实现在插入和删除操作的预期耗时是 O(1)。 

栈在算法中有广泛的应用，比如用于语言解析和运行时的内存管理（“调用栈”）。树或图数 据结构上的深度优先搜索（DFS）是简短而美丽的算法，其中就用到了栈。 

Python中有几种栈实现，每个实现的特性略有不同。下面来分别介绍并比较各自的特性。 

### 5.5.1 列表——简单的内置栈 

Python 的内置列表类型能在正常的 O(1)时间内完成入栈和出栈操作，因此适合作为栈数据 结构。① 

Python的列表在内部以动态数组实现，这意味着在添加或删除时，列表偶尔需要调整元素的 存储空间大小。列表会预先分配一些后备存储空间，因此并非每个入栈或出栈操作都需要调整大 小，所以这些操作的均摊时间复杂度为 O(1)。 

这么做的缺点是列表的性能不如基于链表的实现（如 collections.deque，下面会介绍）， 后者能为插入和删除操作提供稳定的 O(1)时间复杂度。另一方面，列表能在 O(1)时间快速随机 访问堆栈上的元素，这能带来额外的好处。 

使用列表作为堆栈应注意下面几个重要的性能问题。 

为了获得 O(1)的插入和删除性能，必须使用 append()方法将新项添加到列表的末尾，删除 时也要使用 pop()从末尾删除。为了获得最佳性能，基于 Python 列表的栈应该向高索引增长并 向低索引缩小。 

从列表前部添加和删除元素很慢，耗时为O(n)，因为这种情况下必须移动现有元素来为新元 素腾出空间。这是一个性能反模式，应尽可能避免。 

In [172]:
s = []
s.append('eat')
s.append('sleep')
s.append('code')
s

['eat', 'sleep', 'code']

In [173]:
s.pop()

'code'

In [174]:
s.pop()

'sleep'

In [175]:
s.pop()

'eat'

In [176]:
s.pop()

IndexError: pop from empty list

### 5.5.2 collections.deque——快速且稳健的栈 

deque 类实现了一个双端队列，支持在 O(1)时间（非均摊）从两端添加和移除元素。因为 双端队列支持从两端添加和删除元素，所以既可以作为队列也可以作为栈。① 

Python的 deque 对象以双向链表实现，这为插入和删除元素提供了出色且一致的性能，但 是随机访问位于栈中间元素的性能很差，耗时为 O(n)。② 

总之，如果想在 Python 的标准库中寻找一个具有链表性能特征的栈数据结构实现，那么 collections.deque 是不错的选择。 

In [177]:
from collections import deque
s = deque()
s.append('eat')
s.append('sleep')
s.append('code')
s

deque(['eat', 'sleep', 'code'])

In [179]:
s.pop()

'code'

In [180]:
s.pop()

'sleep'

In [181]:
s.pop()

'eat'

In [182]:
s.pop()

IndexError: pop from an empty deque

### 5.5.3 queue.LifoQueue——为并行计算提供锁语义 

queue.LifoQueue 这个位于 Python标准库中的栈实现是同步的，提供了锁语义来支持多个 并发的生产者和消费者。③ 

除了 LifoQueue 之外，queue 模块还包含其他几个类，都实现了用于并行计算的多生产者/ 多用户队列。 

在不同情况下，锁语义即可能会带来帮助，也可能会导致不必要的开销。在后面这种情况下， 好使用 list 或 deque 作为通用栈。 

In [7]:
from queue import LifoQueue
s = LifoQueue()
s.put('eat')
s.put('sleep')
s.put('code')
s

In [8]:
s.get()

'code'

In [9]:
s.get()

'sleep'

In [10]:
s.get()

'eat'

In [11]:
s.get_nowait() 

Empty: 

In [ ]:
s.get()
#阻塞，永远停在这里。。。。。。

### 5.5.4 比较Python中各个栈的实现 

从上面可以看出，Python中有多种栈数据结构的实现，各自的特性稍有区别，在性能和用途 上也各有优劣。 

如果不寻求并行处理支持（或者不想手动处理上锁和解锁），可选择内置列表类型或 collections.deque。两者背后使用的数据结构和总体易用性有所不同。 

 列表底层是动态数组，因此适用于快速随机访问，但在添加或删除元素时偶尔需要调整 大小。列表会预先分配一些备用存储空间，因此不是每个入栈或出栈操作都需要调整大 小，这些操作的均摊时间复杂度为 O(1)。但需要小心，只能用 append()和 pop()从“右 侧”插入和删除元素，否则性能会下降为 O(n)。 

 collections.deque 底层是双向链表，为从两端的添加和删除操作进行了优化，为这 些操作提供了一致的 O(1)性能。collections.deque 不仅性能稳定，而且便于使用， 不必担心在“错误的一端”添加或删除项。

总之，我认为 collections.deque 是在 Python中实现栈（LIFO队列）的绝佳选择。 

### 5.5.5 关键要点 

 Python中有几个栈实现，每种实现的性能和使用特性略有不同。 

 collections.deque 提供安全且快速的通用栈实现。 

 内置列表类型可以作为栈使用，但要小心只能使用 append()和 pop()来添加和删除项， 以避免性能下降。 

## 5.6 队列（先进先出） 

本节将介绍仅使用 Python标准库中的内置数据类型和类来实现 FIFO队列数据结构，首先来 回顾一下什么是队列。 

队列是含有一组对象的容器，支持快速插入和删除的先进先出语义。插入和删除操作有时 称为入队（enqueue）和出队（dequeue）。与列表或数组不同，队列通常不允许随机访问所包含 的对象。 

来看一个先进先出队列在现实中的类比。 

想象在 PyCon注册的第一天，一些 Python高手等着领取会议徽章。新到的人依次 进入会场并排队领取徽章，队列后面会有其他人继续排队。移除动作发生在队列前端， 因为开发者领取徽章和会议礼品袋后就离开了。 

另一种记住队列数据结构特征的方法是将其视为管道。 

新元素（水分子、乒乓球等）从管道一端移向另一端并在那里被移除。当元素在队 列中（想象成位于一根坚固的金属管中）时是无法接触的。唯一能够与队列中元素交互 的方法是在管道后端添加新元素（入队）或在管道前端删除元素（出队）。 

队列与栈类似，但删除元素的方式不同

队列删除的是先添加的项（先进先出），而栈删除的是近添加的项（后进先出）。 

在性能方面，实现合理的队列在插入和删除方面的操作预计耗时为 O(1)。插入和删除是队列 上的两个主要操作，在正确的实现中应该很快。 

队列在算法中有广泛的应用，经常用于解决调度和并行编程问题。在树或图数据结构上进行 宽度优先搜索（BFS）是一种简短而美丽的算法，其中就用到了队列。 

调度算法通常在内部使用优先级队列。这些是特化的队列，其中元素的顺序不是基于插入时 间，而是基于优先级。队列根据元素的键计算到每个元素的优先级。下一节详细介绍优先级队列 以及它们在 Python中的实现方式。 

不过普通队列无法重新排列所包含的元素。就像在管道示例中一样，元素输入和输出的顺序 完全一致。 

Python中实现了几个队列，每种实现的特征略有不同，下面就来看看。 

### 5.6.1 列表——非常慢的队列 

普通列表可以作为队列，但从性能角度来看并不理想。①由于在起始位置插入或删除元素需 要将所有其他元素都移动一个位置，因此需要的时间为 O(n)。 

因此不推荐在 Python中凑合用列表作为队列使用（除非只处理少量元素）： 

In [12]:
q = []
q.append('eat')
q.append('sleep')
q.append('code')
q

['eat', 'sleep', 'code']

In [14]:
#小心，这种操作很慢
q.pop(0)

'eat'

### 5.6.2 collections.deque——快速和稳健的队列 

deque 类实现了一个双端队列，支持在 O(1)时间（非均摊）中从任一端添加和删除元素。 由于 deque 支持从两端添加和移除元素，因此既可用作队列也可用作栈。② 

Python的 deque 对象以双向链表实现。③这为插入和删除元素提供了出色且一致的性能，但 是随机访问位于栈中间元素的性能很差，耗时为 O(n)。 

因此，默认情况下 collections.deque 是 Python标准库中不错的队列型数据结构： 

In [15]:
from collections import deque
q = deque()
q.append('eat')
q.append('sleep')
q.append('code')
q

deque(['eat', 'sleep', 'code'])

In [16]:
q.popleft()

'eat'

In [17]:
q.popleft()

'sleep'

In [18]:
q.popleft()

'code'

In [19]:
q.popleft()

IndexError: pop from an empty deque

### 5.6.3 queue.Queue——为并行计算提供的锁语义 

queue.Queue 在 Python标准库中以同步的方式实现，提供了锁语义来支持多个并发的生产 者和消费者。① 

queue 模块包含其他多个实现多生产者/多用户队列的类，这些队列对并行计算很有用。 

在不同情况下，锁语义可能会带来帮助，也可能会导致不必要的开销。在后面这种情况下， 好使用 collections.deque 作为通用队列： 

In [20]:
from queue import Queue
q = Queue()
q.put('eat')
q.put('sleep')
q.put('code')
q

In [21]:
q.get()

'eat'

In [22]:
q.get()

'sleep'

In [23]:
q.get()

'code'

In [24]:
q.get_nowait()

Empty: 

### 5.6.4 multiprocessing.Queue——共享作业队列 

multiprocessing.Queue 作为共享作业队列来实现，允许多个并发 worker并行处理队列 中的元素。②由于 CPython中存在全局解释器锁（GIL），因此无法在单个解释器进程上执行某些 并行化过程，使得大家都转向基于进程的并行化。 

作为专门用于在进程间共享数据的队列实现，使用 multiprocessing.Queue 能够方便地 在多个进程中分派工作，以此来绕过 GIL的限制。这种类型的队列可以跨进程存储和传输任何可 pickle的对象： 

In [26]:
from multiprocessing import Queue
q = Queue()
q.put('eat')
q.put('sleep')
q.put('code')
q

In [27]:
q.get()

'eat'

In [28]:
q.get()

'sleep'

In [29]:
q.get()

'code'

### 5.6.5 关键要点 

 Python核心语言及其标准库中含有几种队列实现。 

 列表对象可以用作队列，但由于性能较差，通常不建议这么做。

 如果不需要支持并行处理，那么 collections.deque 是 Python中实现 FIFO队列数据 结构的佳选择。collections.deque 是非常优秀的队列实现，具备期望的性能特征， 并且可以用作栈（LIFO队列）。 

## 5.7 优先队列
 

优先队列是一个容器数据结构，使用具有全序关系①的键（例如用数值表示的权重）来管理 元素，以便快速访问容器中键值最小或最大的元素。 

优先队列可被视为队列的改进版，其中元素的顺序不是基于插入时间，而是基于优先级的。 对键进行处理能得到每个元素的优先级。 

优先级队列通常用于处理调度问题，例如优先考虑更加紧急的任务。 

来看看操作系统任务调度器的工作。 

理想情况下，系统上的高优先级任务（如玩实时游戏）级别应高于低优先级的任务 （如在后台下载更新）。优先级队列将待执行的任务根据紧急程度排列，任务调度程序能 够快速选取并优先执行优先级最高的任务。 

本节将介绍如何使用 Python 语言内置或位于标准库中的数据结构来实现优先队列。每种实 现都有各自的优缺点，但其中有一种实现能应对大多数常见情况，下面一起来看看。 

### 5.7.1 列表——手动维护有序队列 

使用有序列表能够快速识别并删除小或大的元素，缺点是向列表插入元素表是很慢的 O(n)操作。 

虽然用标准库中的 bisect.insort①能在 O(logn)时间内找到插入位置，但缓慢的插入操作 才是瓶颈。

向列表添加并重新排序来维持顺序也至少需要 O(nlogn)的时间。另一个缺点是在插入新元素 时，必须手动重新排列列表。缺少这一步就很容易引入 bug，因此担子总是压在开发人员身上。 

因此，有序列表只适合在插入次数很少的情况下充当优先队列。 

In [32]:
q = []
q.append((2,'code'))
q.append((1,'eat'))
q.append((3,'sleep'))
# 注意：每当添加新元素或调用 bisect.insort()时，都要重新排序。 
q.sort(reverse = True)

while q:
    next_item = q.pop()
    print(next_item)

(1, 'eat')
(2, 'code')
(3, 'sleep')


### 5.7.2 heapq——基于列表的二叉堆 

heapq 是二叉堆，通常用普通列表实现，能在 O(logn)时间内插入和获取小的元素。② 

heapq 模块是在 Python中不错的优先级队列实现。由于 heapq在技术上只提供小堆实现， 因此必须添加额外步骤来确保排序稳定性，以此来获得“实际”的优先级队列中所含有的预期 特性。③ 

In [33]:
import heapq
q = []
heapq.heappush(q,(2,'code'))
heapq.heappush(q,(1,'eat'))
heapq.heappush(q,(3,'sleep'))

while q:
    next_item = heapq.heappop(q)
    print(next_item)

(1, 'eat')
(2, 'code')
(3, 'sleep')


### 5.7.3 queue.PriorityQueue——美丽的优先级队列 

queue.PriorityQueue 这个优先级队列的实现在内部使用了 heapq，时间和空间复杂度与 heapq 相同。① 

区别在于 PriorityQueue 是同步的，提供了锁语义来支持多个并发的生产者和消费者。 

在不同情况下，锁语义可能会带来帮助，也可能会导致不必要的开销。不管哪种情况，你都 可能更喜欢 PriorityQueue 提供的基于类的接口，而不是使用 heapq 提供的基于函数的接口

In [35]:
from queue import PriorityQueue 
 
q = PriorityQueue() 
 
q.put((2, 'code')) 
q.put((1, 'eat'))
q.put((3, 'sleep')) 
 
while not q.empty():     
    next_item = q.get()     
    print(next_item) 

(1, 'eat')
(2, 'code')
(3, 'sleep')


### 5.7.4 关键要点 

 Python提供了几种优先队列实现可以使用。 

 queue.PriorityQueue 是其中的首选，具有良好的面向对象的接口，从名称就能明白 其用途

 如果想避免 queue.PriorityQueue 的锁开销，那么建议直接使用 heapq 模块。 